In [181]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

In [323]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import geopandas as gpd

graph = Graph.from_file("PA/full/PA_VTD.shp")




In [183]:
elections = [
    Election("SEN10", {"Democratic": "SEN10D", "Republican": "SEN10R"}),
    Election("SEN12", {"Democratic": "USS12D", "Republican": "USS12R"}),
    Election("SEN16", {"Democratic": "T16SEND", "Republican": "T16SENR"}),
    Election("PRES12", {"Democratic": "PRES12D", "Republican": "PRES12R"}),
    Election("PRES16", {"Democratic": "T16PRESD", "Republican": "T16PRESR", "Other": "T16PRESOTH"})
]

In [225]:

# Population updater, for computing how close to equality the district
# populations are. "TOT_POP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TOT_POP", alias="population"), 
                "whites": updaters.Tally("WHITE_POP", alias="whites"),
                "dem": updaters.Tally("T16PRESD", alias="dem"),
                "rep": updaters.Tally("T16PRESR", alias="rep"),
                "oth": updaters.Tally("T16PRESOTH", alias="oth"), 
                "d12": updaters.Tally("PRES12D", alias = "d12"), 
                "r12": updaters.Tally("PRES12R", alias = "r12"), 
                "o12": updaters.Tally("PRES12O", alias = "o12")}

# Election updaters, for computing election results using the vote totals
# from our shapefile.
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

In [329]:
initial_partition = GeographicPartition(
    graph,
    assignment="REMEDIAL_P",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOT_POP", alias="population"),
        "whites": Tally("WHITE_POP", alias="whites"),
        "dem": Tally("T16PRESD", alias="dem"),
        "rep": Tally("T16PRESR", alias="rep"),
        "oth": Tally("T16PRESOTH", alias="oth"),
        "PRES16": election
    }
)


In [330]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.03)

In [332]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        pop_constraint,
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=1002
)

In [331]:
# The ReCom proposal needs to know the ideal population for the districts so that
# we can improve speed by bailing early on unbalanced partitions.

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

In [322]:
j = 1 

for partition in chain: 
    
    if j%10 == 0: 
        shp = "./IA/" + str(int(j/10)) + "/districts.shp"
        ft = "./IA/" + str(int(j/10)) + "/features.csv"
        assignment = {}
        for i in partition.assignment: 
            assignment[i] = partition.assignment[i]
        df2 = df.copy(deep = True)
        print("Copied", j)
        df2["CD"] = df2.index.map(assignment)
        print("Mapped", j)
        gf = df2.dissolve(by='CD')
        gf["CD"] = gf.index
        print("Dissolved", j)
        gf.to_file(shp) 
        d = {}
        print("Features", j)
        for i in range(1, 5): 
            d[i] = {}
            d[i]["Population"] = partition["population"][i]
            d[i]["White"] = partition["whites"][i]
            d[i]["Democratic"] = partition["dem"][i]
            d[i]["Republican"] = partition["rep"][i]
            d[i]["Other"] = partition["oth"][i]

        dft = pd.DataFrame(d).T
        dft.to_csv(ft)
        print("feat!!!")
    print(j)
    j += 1
    


1
2
3
4
5
6
7
8
9
Copied 10
Mapped 10
Dissolved 10
Features 10
feat!!!
10
11
12
13
14
15
16
17
18
19
Copied 20
Mapped 20
Dissolved 20
Features 20
feat!!!
20
21
22
23
24
25
26
27
28
29
Copied 30
Mapped 30
Dissolved 30
Features 30
feat!!!
30
31
32
33
34
35
36
37
38
39
Copied 40
Mapped 40
Dissolved 40
Features 40
feat!!!
40
41
42
43
44
45
46
47
48
49
Copied 50
Mapped 50
Dissolved 50
Features 50
feat!!!
50
51
52
53
54
55
56
57
58
59
Copied 60
Mapped 60
Dissolved 60
Features 60
feat!!!
60
61
62
63
64
65
66
67
68
69
Copied 70
Mapped 70
Dissolved 70
Features 70
feat!!!
70
71
72
73
74
75
76
77
78
79
Copied 80
Mapped 80
Dissolved 80
Features 80
feat!!!
80
81
82
83
84
85
86
87
88
89
Copied 90
Mapped 90
Dissolved 90
Features 90
feat!!!
90
91
92
93
94
95
96
97
98
99
Copied 100
Mapped 100
Dissolved 100
Features 100
feat!!!
100
101
102
103
104
105
106
107
108
109
Copied 110
Mapped 110
Dissolved 110
Features 110
feat!!!
110
111
112
113
114
115
116
117
118
119
Copied 120
Mapped 120
Dissolved 120
Featu

866
867
868
869
Copied 870
Mapped 870
Dissolved 870
Features 870
feat!!!
870
871
872
873
874
875
876
877
878
879
Copied 880
Mapped 880
Dissolved 880
Features 880
feat!!!
880
881
882
883
884
885
886
887
888
889
Copied 890
Mapped 890
Dissolved 890
Features 890
feat!!!
890
891
892
893
894
895
896
897
898
899
Copied 900
Mapped 900
Dissolved 900
Features 900
feat!!!
900
901
902
903
904
905
906
907
908
909
Copied 910
Mapped 910
Dissolved 910
Features 910
feat!!!
910
911
912
913
914
915
916
917
918
919
Copied 920
Mapped 920
Dissolved 920
Features 920
feat!!!
920
921
922
923
924
925
926
927
928
929
Copied 930
Mapped 930
Dissolved 930
Features 930
feat!!!
930
931
932
933
934
935
936
937
938
939
Copied 940
Mapped 940
Dissolved 940
Features 940
feat!!!
940
941
942
943
944
945
946
947
948
949
Copied 950
Mapped 950
Dissolved 950
Features 950
feat!!!
950
951
952
953
954
955
956
957
958
959
Copied 960
Mapped 960
Dissolved 960
Features 960
feat!!!
960
961
962
963
964
965
966
967
968
969
Copied 970
Map

In [175]:
qqqq

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.35545,0.383946,0.388876,0.396026,0.441266,0.456416,0.482548,0.483733,0.514249,0.525005,0.528905,0.547777,0.583928,0.58943,0.656902,0.671029,0.797758,0.926351
1,0.35545,0.383946,0.388876,0.396026,0.441266,0.456416,0.482548,0.483733,0.514249,0.525005,0.528905,0.547777,0.579445,0.58943,0.656902,0.671029,0.801968,0.926351


In [147]:
assignment = {}
for i in initial_partition.assignment: 
    assignment[i] = initial_partition.assignment[i]

In [269]:
df

,Democratic,Other,Population,Republican,White
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,0,0,0,0,0
10,0,0,0,0,0


In [325]:
election = Election("PRES16", {"Dem": "PRES16D", "Rep": "PRES16R"})

initial_partition = Partition(
    graph,
    assignment="REMEDIAL_P",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOT_POP", alias="population"),
        "whites": Tally("WHITE_POP", alias="whites"),
        "dem": Tally("T16PRESD", alias="dem"),
        "rep": Tally("T16PRESR", alias="rep"),
        "oth": Tally("T16PRESOTH", alias="oth"),
        "PRES16": election
    }
)

In [327]:
d = {}

for i in range(1, 19): 
    d[i] = {}
    d[i]["Population"] = initial_partition["population"][i]
    d[i]["White"] = initial_partition["whites"][i]
    d[i]["Democratic"] = initial_partition["dem"][i]
    d[i]["Republican"] = initial_partition["rep"][i]
    d[i]["Other"] = initial_partition["oth"][i]

    
dft = pd.DataFrame(d).T
dft.to_csv("PA/d18/features.csv")

In [272]:
initial_partition["dem"]

defaultdict(int,
            {'06': 141655,
             '14': 226384,
             '13': 181223,
             '10': 108899,
             '01': 133585,
             '09': 183085,
             '05': 163032,
             '04': 113475,
             '11': 177143,
             '02': 132312,
             '08': 164433,
             '12': 205662,
             '03': 147337,
             '07': 131843,
             1: 0,
             2: 0,
             3: 0,
             4: 0,
             5: 0,
             6: 0,
             7: 0,
             8: 0,
             9: 0,
             10: 0,
             11: 0,
             12: 0,
             13: 0,
             14: 0,
             15: 0,
             16: 0,
             17: 0,
             18: 0})

In [111]:
d = {}

for i in range(1, 19): 
    k = str(i)
    print(i, initial_partition["dem"][k]/(initial_partition["dem"][k]+initial_partition["rep"][k]))
    d[i] = {}
    d[i]["Population"] = initial_partition["population"][k]
    d[i]["White"] = initial_partition["whites"][k]
    d[i]["Democratic"] = initial_partition["dem"][k]
    d[i]["Republican"] = initial_partition["rep"][k]
    d[i]["Other"] = initial_partition["oth"][k]

1 0.3938847437694732
2 0.27532424415205736
3 0.4626706196225345
4 0.47598284960422166
5 0.3390988168423019
6 0.3128687509877259
7 0.4675712852844247
8 0.5347361264280437
9 0.5431545556736115
10 0.27535386349322105
11 0.4388201622631691
12 0.3763702204903254
13 0.3349807261770498
14 0.6243530668070976
15 0.509111462423717
16 0.6791887507877962
17 0.7786997892752903
18 0.9160877584378866


In [90]:
d

{}

In [45]:
initial_partition.parts[1]

frozenset({48,
           49,
           50,
           953,
           954,
           955,
           956,
           957,
           958,
           959,
           960,
           961,
           962,
           963,
           964,
           965,
           966,
           967,
           968,
           969,
           970,
           971,
           972,
           973,
           974,
           975,
           976,
           977,
           978,
           979,
           980,
           981,
           982,
           983,
           984,
           985,
           986,
           987,
           988,
           989,
           990,
           991,
           992,
           993,
           994,
           995,
           996,
           997,
           998,
           999,
           1000,
           1001,
           1002,
           1003,
           1004,
           1005,
           1006,
           1007,
           1008,
           1009,
           1010,
           1011,

In [52]:
districts = {}
for i in range(1, 19): 
    districts[i] = []
    for k in initial_partition.parts[i]:
        districts[i].append(k)

In [328]:
df

,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,TOTPOP,...,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD,CON,geometry
0,19,127,19127,Marshall,Marshall County,1482770678,1803086,+42.0416910,-092.9814523,40648,...,10257,8472,335,17980,7652,9146,1182,1,1,"POLYGON ((-92.76679299999999 42.12346, -92.766..."
1,19,011,19011,Benton,Benton County,1855117342,5760770,+42.0925474,-092.0576300,26076,...,6862,6940,221,13844,4678,8232,934,1,1,"POLYGON ((-91.947732 41.861856, -91.955139 41...."
2,19,041,19041,Clay,Clay County,1469139214,13866941,+43.0798220,-095.1497261,16667,...,3385,4951,166,8617,2249,5877,491,4,4,"POLYGON ((-95.269263 43.255373, -95.2613959999..."
3,19,165,19165,Shelby,Shelby County,1530110414,1486135,+41.6790143,-095.3089173,12167,...,2469,3911,103,6370,1662,4362,346,4,4,"POLYGON ((-95.209023 41.863709, -95.2089029999..."
4,19,043,19043,Clayton,Clayton County,2016405612,36586071,+42.8409979,-091.3235108,18129,...,4806,4164,168,9129,3237,5317,575,1,1,"POLYGON ((-91.25080199999999 42.645576, -91.25..."
5,19,097,19097,Jackson,Jackson County,1647332960,35141529,+42.1642812,-090.5745965,19848,...,5907,4177,158,10310,3837,5824,649,1,1,"POLYGON ((-90.89822599999999 42.120593, -90.89..."
6,19,079,19079,Hamilton,Hamilton County,1493777716,1965344,+42.3907681,-093.7091980,15673,...,3782,3991,154,7694,2726,4463,505,4,4,"POLYGON ((-93.81572899999999 42.209597, -93.82..."
7,19,173,19173,Taylor,Taylor County,1377618597,7133648,+40.7379489,-094.6971082,6317,...,1262,1683,50,3029,758,2111,160,3,3,"POLYGON ((-94.47147 40.725934, -94.47147 40.72..."
8,19,139,19139,Muscatine,Muscatine County,1133040484,30198875,+41.4837760,-091.1186992,42745,...,11323,8168,374,19434,8368,9584,1482,2,2,"POLYGON ((-91.368521 41.423178, -91.3685199999..."
9,19,197,19197,Wright,Wright County,1503286817,4703964,+42.7330073,-093.7347352,13229,...,2836,3349,93,6026,1896,3800,330,4,4,"POLYGON ((-93.97146499999999 42.8197, -93.9714..."


In [54]:
with open("assignment.json", 'w') as outfile:
    json.dump(districts, outfile)

In [49]:
import json

In [55]:
graph

<Graph [8921 nodes, 25228 edges]>

In [56]:
graph[1]

AtlasView({2079: {'shared_perim': 8213.240384992361}, 2107: {'shared_perim': 3829.504455218072}, 257: {'shared_perim': 6321.734123390416}, 2063: {'shared_perim': 7854.594090375218}, 6: {'shared_perim': 2338.8949900079633}, 247: {'shared_perim': 11196.022380107901}})

In [63]:
graph[0]

AtlasView({272: {'shared_perim': 6816.47843648668}, 273: {'shared_perim': 816.7154716142254}, 264: {'shared_perim': 1673.5980244719674}, 261: {'shared_perim': 15370.482333044538}, 262: {'shared_perim': 14218.781760578944}, 173: {'shared_perim': 2017.2944638176064}, 178: {'shared_perim': 4939.420849486653}})

In [78]:
len(df)

8921

In [127]:
initial_partition.assignment[0]

14

In [137]:
df

,Democratic,Other,Population,Republican,White
1,109858.0,12194.0,667814.0,169051.0,606320.0
2,76294.0,10849.0,667925.0,200812.0,629711.0
3,140297.0,9935.0,707523.0,162936.0,601337.0
4,144318.0,11414.0,707376.0,158882.0,543339.0
5,94379.0,10341.0,669194.0,183944.0,619284.0
6,95027.0,11454.0,677661.0,208701.0,645161.0
7,163159.0,15713.0,733126.0,185791.0,601365.0
8,199771.0,15518.0,735329.0,173817.0,608246.0
9,211638.0,14473.0,740088.0,178008.0,616451.0
10,83066.0,11554.0,674789.0,218604.0,625680.0
